# Aggregate analysis of pileups / consensus sequences from viral deep sequencing

Import Python modules:

In [ ]:
import altair as alt

import pandas as pd

Get variables from `snakemake`:


######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/fh/fast/bloom_j/software/miniconda3/envs/early_Wuhan_SARS-CoV-2/lib/python3.8/site-packages', '/home/jbloom/early_Wuhan_SARS-CoV-2/notebooks']); import pickle; snakemake = pickle.loads(b'\x80\x04\x95\xc0&\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8c5results/pileup/Wuhan-Hu-1_Wu_et_al/diffs_from_ref.csv\x94\x8c2results/pileup/WIV02_Zhou_et_al/diffs_from_ref.csv\x94\x8c5results/pileup/WIV02-01_Zhou_et_al/diffs_from_ref.csv\x94\x8c5results/pileup/WIV02-02_Zhou_et_al/diffs_from_ref.csv\x94\x8c2results/pileup/WIV04_Zhou_et_al/diffs_from_ref.csv\x94\x8c5results/pileup/WIV04-01_Zhou_et_al/diffs_from_ref.csv\x94\x8c5results/pileup/WIV04-02_Zhou_et_al/diffs_from_ref.csv\x94\x8c2results/pileup/WIV05_Zhou_et_al/diffs_from_ref.csv\x94\x8c2results/pileup/WIV06_Zhou_et_al/diffs_from_ref.csv\x94\x8c5results/pileup/WIV06-01_Zhou_et_al/diffs_from_ref.csv\x94\x8c5results/pileup/WIV06-02_Zhou_et_al/diffs_from_ref.csv\x94\x8c2results/pileup/WIV07_Zhou_et_al/diffs_from_ref.csv\x94\x8c5results/pileup/WIV07-01_Zhou_et_al/diffs_from_ref.csv\x94\x8c5results/pileup/WIV07-02_Zhou_et_al/diffs_from_ref.csv\x94\x8c2results/pileup/WHU01_Chen_et_al/diffs_from_ref.csv\x94\x8c2results/pileup/WHU02_Chen_et_al/diffs_from_ref.csv\x94\x8c=results/pileup/Nepal-student_Bastola_et_al/diffs_from_ref.csv\x94\x8c2results/pileup/HBCDC-HB-01_2019/diffs_from_ref.csv\x94\x8c2results/pileup/HBCDC-HB-02_2019/diffs_from_ref.csv\x94\x8c2results/pileup/HBCDC-HB-03_2019/diffs_from_ref.csv\x94\x8c2results/pileup/HBCDC-HB-04_2019/diffs_from_ref.csv\x94\x8c2results/pileup/HBCDC-HB-01_2020/diffs_from_ref.csv\x94\x8c2results/pileup/HBCDC-HB-02_2020/diffs_from_ref.csv\x94\x8c2results/pileup/HBCDC-HB-03_2020/diffs_from_ref.csv\x94\x8c2results/pileup/HBCDC-HB-04_2020/diffs_from_ref.csv\x94\x8c2results/pileup/HBCDC-HB-06_2020/diffs_from_ref.csv\x94\x8c2results/pileup/HBCDC-HB-07_2020/diffs_from_ref.csv\x94\x8c4results/pileup/Wuhan-Hu-1_Wu_et_al/frac_coverage.csv\x94\x8c1results/pileup/WIV02_Zhou_et_al/frac_coverage.csv\x94\x8c4results/pileup/WIV02-01_Zhou_et_al/frac_coverage.csv\x94\x8c4results/pileup/WIV02-02_Zhou_et_al/frac_coverage.csv\x94\x8c1results/pileup/WIV04_Zhou_et_al/frac_coverage.csv\x94\x8c4results/pileup/WIV04-01_Zhou_et_al/frac_coverage.csv\x94\x8c4results/pileup/WIV04-02_Zhou_et_al/frac_coverage.csv\x94\x8c1results/pileup/WIV05_Zhou_et_al/frac_coverage.csv\x94\x8c1results/pileup/WIV06_Zhou_et_al/frac_coverage.csv\x94\x8c4results/pileup/WIV06-01_Zhou_et_al/frac_coverage.csv\x94\x8c4results/pileup/WIV06-02_Zhou_et_al/frac_coverage.csv\x94\x8c1results/pileup/WIV07_Zhou_et_al/frac_coverage.csv\x94\x8c4results/pileup/WIV07-01_Zhou_et_al/frac_coverage.csv\x94\x8c4results/pileup/WIV07-02_Zhou_et_al/frac_coverage.csv\x94\x8c1results/pileup/WHU01_Chen_et_al/frac_coverage.csv\x94\x8c1results/pileup/WHU02_Chen_et_al/frac_coverage.csv\x94\x8c<results/pileup/Nepal-student_Bastola_et_al/frac_coverage.csv\x94\x8c1results/pileup/HBCDC-HB-01_2019/frac_coverage.csv\x94\x8c1results/pileup/HBCDC-HB-02_2019/frac_coverage.csv\x94\x8c1results/pileup/HBCDC-HB-03_2019/frac_coverage.csv\x94\x8c1results/pileup/HBCDC-HB-04_2019/frac_coverage.csv\x94\x8c1results/pileup/HBCDC-HB-01_2020/frac_coverage.csv\x94\x8c1results/pileup/HBCDC-HB-02_2020/frac_coverage.csv\x94\x8c1results/pileup/HBCDC-HB-03_2020/frac_coverage.csv\x94\x8c1results/pileup/HBCDC-HB-04_2020/frac_coverage.csv\x94\x8c1results/pileup/HBCDC-HB-06_2020/frac_coverage.csv\x94\x8c1results/pileup/HBCDC-HB-07_2020/frac_coverage.csv\x94e}\x94(\x8c\x06_names\x94}\x94(\x8c\x0ediffs_from_ref\x94K\x00K\x1b\x86\x94\x8c\rfrac_coverage\x94K\x1bK6\x86\x94u\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94ehI\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(hO)}\x94\x8c\x05_name\x94hIsNt\x94bhJhMhO\x85\x94R\x94(hO)}\x94hShJsNt\x94bhCh\x06\x8c\tNamedlist\x94\x93\x94)\x81\x94(h\nh\x0bh\x0ch\rh\x0eh\x0fh\x10h\x11h\x12h\x13h\x14h\x15h\x16h\x17h\x18h\x19h\x1ah\x1bh\x1ch\x1dh\x1eh\x1fh h!h"h#h$e}\x94(hA}\x94hG]\x94(hIhJehIhMhO\x85\x94R\x94(hO)}\x94hShIsNt\x94bhJhMhO\x85\x94R\x94(hO)}\x94hShJsNt\x94bubhEhZ)\x81\x94(h%h&h\'h(h)h*h+h,h-h.h/h0h1h2h3h4h5h6h7h8h9h:h;h<h=h>h?e}\x94(hA}\x94hG]\x94(hIhJehIhMhO\x85\x94R\x94(hO)}\x94hShIsNt\x94bhJhMhO\x85\x94R\x94(hO)}\x94hShJsNt\x94bubub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94(\x8c results/pileup/frac_coverage.csv\x94\x8c!results/pileup/frac_coverage.html\x94e}\x94(hA}\x94(\x8c\x13frac_coverage_stats\x94K\x00N\x86\x94\x8c\x13frac_coverage_chart\x94K\x01N\x86\x94uhG]\x94(hIhJehIhMhO\x85\x94R\x94(hO)}\x94hShIsNt\x94bhJhMhO\x85\x94R\x94(hO)}\x94hShJsNt\x94bh{hwh}hxub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94}\x94(\x8c\x13Wuhan-Hu-1_Wu_et_al\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR10971381\x94a\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2008-3\x94\x8c\x07genbank\x94\x8c\x08MN908947\x94\x8c\x03sex\x94\x8c\x04male\x94\x8c\x0fcollection_date\x94\x8c\x08datetime\x94\x8c\x04date\x94\x93\x94C\x04\x07\xe4\x01\x02\x94\x85\x94R\x94u\x8c\x10WIV02_Zhou_et_al\x94}\x94(\x8c\naccessions\x94]\x94(\x8c\x0bSRR11092058\x94\x8c\x0bSRR11092063\x94e\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2012-7\x94\x8c\x07genbank\x94\x8c\x08MN996527\x94\x8c\x03sex\x94\x8c\x04male\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x13WIV02-01_Zhou_et_al\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11092058\x94a\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2012-7\x94\x8c\x07genbank\x94\x8c\x08MN996527\x94\x8c\x03sex\x94\x8c\x04male\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x13WIV02-02_Zhou_et_al\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11092063\x94a\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2012-7\x94\x8c\x07genbank\x94\x8c\x08MN996527\x94\x8c\x03sex\x94\x8c\x04male\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x10WIV04_Zhou_et_al\x94}\x94(\x8c\naccessions\x94]\x94(\x8c\x0bSRR11092057\x94\x8c\x0bSRR11092062\x94e\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2012-7\x94\x8c\x07genbank\x94\x8c\x08MN996528\x94\x8c\x03sex\x94\x8c\x06female\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x13WIV04-01_Zhou_et_al\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11092057\x94a\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2012-7\x94\x8c\x07genbank\x94\x8c\x08MN996528\x94\x8c\x03sex\x94\x8c\x06female\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x13WIV04-02_Zhou_et_al\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11092062\x94a\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2012-7\x94\x8c\x07genbank\x94\x8c\x08MN996528\x94\x8c\x03sex\x94\x8c\x06female\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x10WIV05_Zhou_et_al\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11092061\x94a\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2012-7\x94\x8c\x07genbank\x94\x8c\x08MN996529\x94\x8c\x03sex\x94\x8c\x06female\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x10WIV06_Zhou_et_al\x94}\x94(\x8c\naccessions\x94]\x94(\x8c\x0bSRR11092056\x94\x8c\x0bSRR11092060\x94e\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2012-7\x94\x8c\x07genbank\x94\x8c\x08MN996530\x94\x8c\x03sex\x94\x8c\x04male\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x13WIV06-01_Zhou_et_al\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11092056\x94a\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2012-7\x94\x8c\x07genbank\x94\x8c\x08MN996530\x94\x8c\x03sex\x94\x8c\x04male\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x13WIV06-02_Zhou_et_al\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11092060\x94a\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2012-7\x94\x8c\x07genbank\x94\x8c\x08MN996530\x94\x8c\x03sex\x94\x8c\x04male\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x10WIV07_Zhou_et_al\x94}\x94(\x8c\naccessions\x94]\x94(\x8c\x0bSRR11092064\x94\x8c\x0bSRR11092059\x94e\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2012-7\x94\x8c\x07genbank\x94\x8c\x08MN996531\x94\x8c\x03sex\x94\x8c\x04male\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x13WIV07-01_Zhou_et_al\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11092064\x94a\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2012-7\x94\x8c\x07genbank\x94\x8c\x08MN996531\x94\x8c\x03sex\x94\x8c\x04male\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x13WIV07-02_Zhou_et_al\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11092059\x94a\x8c\tstudy_url\x94\x8c1https://www.nature.com/articles/s41586-020-2012-7\x94\x8c\x07genbank\x94\x8c\x08MN996531\x94\x8c\x03sex\x94\x8c\x04male\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x10WHU01_Chen_et_al\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR10903402\x94a\x8c\tstudy_url\x94\x8cBhttps://www.tandfonline.com/doi/full/10.1080/22221751.2020.1725399\x94\x8c\x07genbank\x94\x8c\x08MN988668\x94\x8c\x03sex\x94\x8c\x04male\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe4\x01\x02\x94\x85\x94R\x94u\x8c\x10WHU02_Chen_et_al\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR10903401\x94a\x8c\tstudy_url\x94\x8cBhttps://www.tandfonline.com/doi/full/10.1080/22221751.2020.1725399\x94\x8c\x07genbank\x94\x8c\x08MN988669\x94\x8c\x03sex\x94\x8c\x06female\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe4\x01\x02\x94\x85\x94R\x94u\x8c\x1bNepal-student_Bastola_et_al\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11177792\x94a\x8c\tstudy_url\x94\x8cShttps://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30067-0/fulltext\x94\x8c\x07genbank\x94\x8c\x08MT072688\x94\x8c\x03sex\x94\x8c\x04male\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe4\x01\r\x94\x85\x94R\x94u\x8c\x10HBCDC-HB-01_2019\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11454614\x94a\x8c\tstudy_url\x94\x8c/https://www.ncbi.nlm.nih.gov/biosample/14479128\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x10HBCDC-HB-02_2019\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11454615\x94a\x8c\tstudy_url\x94\x8c/https://www.ncbi.nlm.nih.gov/biosample/14479127\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x10HBCDC-HB-03_2019\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11454613\x94a\x8c\tstudy_url\x94\x8c/https://www.ncbi.nlm.nih.gov/biosample/14479129\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x10HBCDC-HB-04_2019\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11454612\x94a\x8c\tstudy_url\x94\x8c/https://www.ncbi.nlm.nih.gov/biosample/14479130\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe3\x0c\x1e\x94\x85\x94R\x94u\x8c\x10HBCDC-HB-01_2020\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11454608\x94a\x8c\tstudy_url\x94\x8c/https://www.ncbi.nlm.nih.gov/biosample/14479134\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe4\x01\x08\x94\x85\x94R\x94u\x8c\x10HBCDC-HB-02_2020\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11454609\x94a\x8c\tstudy_url\x94\x8c/https://www.ncbi.nlm.nih.gov/biosample/14479133\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe4\x01\x11\x94\x85\x94R\x94u\x8c\x10HBCDC-HB-03_2020\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11454610\x94a\x8c\tstudy_url\x94\x8c/https://www.ncbi.nlm.nih.gov/biosample/14479132\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe4\x01\x12\x94\x85\x94R\x94u\x8c\x10HBCDC-HB-04_2020\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11454611\x94a\x8c\tstudy_url\x94\x8c/https://www.ncbi.nlm.nih.gov/biosample/14479131\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe4\x01\x12\x94\x85\x94R\x94u\x8c\x10HBCDC-HB-06_2020\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11454607\x94a\x8c\tstudy_url\x94\x8c/https://www.ncbi.nlm.nih.gov/biosample/14479135\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe4\x02\x07\x94\x85\x94R\x94u\x8c\x10HBCDC-HB-07_2020\x94}\x94(\x8c\naccessions\x94]\x94\x8c\x0bSRR11454606\x94a\x8c\tstudy_url\x94\x8c/https://www.ncbi.nlm.nih.gov/biosample/14479136\x94\x8c\x0fcollection_date\x94h\x9bC\x04\x07\xe4\x02\x08\x94\x85\x94R\x94uua}\x94(hA}\x94\x8c\x07samples\x94K\x00N\x86\x94shG]\x94(hIhJehIhMhO\x85\x94R\x94(hO)}\x94hShIsNt\x94bhJhMhO\x85\x94R\x94(hO)}\x94hShJsNt\x94bj\x03\x02\x00\x00h\x8cub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94}\x94(hA}\x94hG]\x94(hIhJehIhMhO\x85\x94R\x94(hO)}\x94hShIsNt\x94bhJhMhO\x85\x94R\x94(hO)}\x94hShJsNt\x94bub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01e}\x94(hA}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94uhG]\x94(hIhJehIhMhO\x85\x94R\x94(hO)}\x94hShIsNt\x94bhJhMhO\x85\x94R\x94(hO)}\x94hShJsNt\x94bj$\x02\x00\x00K\x01j&\x02\x00\x00K\x01ub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8c,results/logs/aggregate_pileup_analysis.ipynb\x94a}\x94(hA}\x94\x8c\x08notebook\x94K\x00N\x86\x94shG]\x94(hIhJehIhMhO\x85\x94R\x94(hO)}\x94hShIsNt\x94bhJhMhO\x85\x94R\x94(hO)}\x94hShJsNt\x94bj8\x02\x00\x00j5\x02\x00\x00ub\x8c\x06config\x94}\x94(\x8c\x08max_cpus\x94K\x04\x8c\x0bscratch_dir\x94\x8c\x11results/_scratch/\x94\x8c\x1cper_sample_pileups_in_report\x94\x89\x8c\x07genomes\x94}\x94\x8c\nWuhan-Hu-1\x94}\x94(\x8c\x05fasta\x94\x8c}ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/009/858/895/GCF_009858895.2_ASM985889v3/GCF_009858895.2_ASM985889v3_genomic.fna.gz\x94\x8c\x04name\x94\x8c\x0bNC_045512.2\x94us\x8c\x12genome_trim3_polyA\x94\x88\x8c\x12comparator_genomes\x94}\x94(\x8c\x06RaTG13\x94}\x94\x8c\x07genbank\x94\x8c\nMN996532.2\x94s\x8c\x11GD_Pangolin_MP789\x94}\x94\x8c\x07genbank\x94\x8c\nMT121216.1\x94s\x8c\x08RacCS203\x94}\x94\x8c\x07genbank\x94\x8c\nMW251308.1\x94su\x8c\x0chost_genomes\x94}\x94\x8c\x05human\x94}\x94(\x8c\x05fasta\x94\x8cjftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz\x94\x8c\x0fsex_chromosomes\x94}\x94(\x8c\x04male\x94\x8c\x01Y\x94\x8c\x06female\x94\x8c\x01X\x94uus\x8c\x08aligners\x94]\x94(\x8c\x05bbmap\x94\x8c\x08bwa-mem2\x94e\x8c\x04minq\x94K\x14\x8c\x16consensus_min_coverage\x94K\x04\x8c\x12consensus_min_frac\x94G?\xe8\x00\x00\x00\x00\x00\x00\x8c\x14report_frac_coverage\x94]\x94(K\x01K\x02K\x04K\nK\x14K2Kde\x8c\x07samples\x94h\x8cu\x8c\x04rule\x94\x8c\x19aggregate_pileup_analysis\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8c-/home/jbloom/early_Wuhan_SARS-CoV-2/notebooks\x94ub.'); from snakemake.logging import logger; logger.printshellcmds = False; import os; os.chdir('/home/jbloom/early_Wuhan_SARS-CoV-2');
######## snakemake preamble end #########


In [ ]:
diffs_from_ref_files = snakemake.input.diffs_from_ref
frac_coverage_files = snakemake.input.frac_coverage
frac_coverage_stats_file = snakemake.output.frac_coverage_stats
frac_coverage_chart_file = snakemake.output.frac_coverage_chart
samples = snakemake.params.samples

Read in frac coverage for all samples:

In [ ]:
frac_coverage = (
    pd.concat([pd.read_csv(f).assign(sample=sample)
               for f, sample in zip(frac_coverage_files, samples)],
              ignore_index=True)
    )

frac_coverage = frac_coverage[['sample', *frac_coverage.columns.tolist()[: -1]]]

print(f"Writing to {frac_coverage_stats_file}")

frac_coverage.to_csv(frac_coverage_stats_file, float_format='%.4g', index=False)

frac_coverage

Plot frac coverage for all samples:

In [ ]:
cutoff_selection = alt.selection_multi(
        fields=['depth_cutoff'],
        bind='legend',
        init=[{'depth_cutoff': frac_coverage['depth_cutoff'][0]}]
        )

aligner_selection = alt.selection_single(
        name='read',
        fields=['aligner'],
        bind=alt.binding_select(options=frac_coverage['aligner'].unique()),
        init={'aligner': frac_coverage['aligner'][0]}
        )

genome_selection = alt.selection_single(
        name='reference',
        fields=['genome'],
        bind=alt.binding_select(options=frac_coverage['genome'].unique()),
        init={'genome': frac_coverage['genome'][0]}
        )

frac_coverage_chart = (
    alt.Chart(frac_coverage)
    .encode(x=alt.X('frac_above_cutoff:Q',
                    scale=alt.Scale(domain=(0, 1))
                    ),
            y=alt.Y('sample:N'),
            color=alt.condition(cutoff_selection, 'depth_cutoff:N', alt.value(None)),
            tooltip=['sample',
                     'depth_cutoff',
                     alt.Tooltip('frac_above_cutoff', format='.4g'),
                     'sites_above_cutoff',
                     ]
            )
    .mark_point(filled=True,
                size=75)
    .add_selection(cutoff_selection,
                   aligner_selection,
                   genome_selection)
    .transform_filter(aligner_selection)
    .transform_filter(genome_selection)
    .properties(title={'text': 'Fraction of sites exceeding depth cutoff',
                       'subtitle': 'Use (shift) click to select depth cutoffs'})
    )

print(f"Saving chart to {frac_coverage_chart_file}")

frac_coverage_chart.save(frac_coverage_chart_file)

frac_coverage_chart